# 财务数据与因子分析策略2-多因子策略介绍

## 简单因子相关性

* classwork 1

把上节课，简单多因子分析方法所有代码简化并整理到一起。然后修改参数尝试分析其他财务因子。


In [3]:
from scipy.stats.mstats import winsorize
from scipy.stats import pearsonr,spearmanr,kendalltau
import pandas as pd
import numpy as np

from gm.api import *
set_token("eea4b28a3ceb7048603d388bb777ebc67de47385")

In [10]:
def standardize_series(series):#标准化
    std = series.std()
    mean = series.mean()
    return (series-mean)/std

index_symbol='SHSE.000300'
table_caiwu='trading_derivative_indicator'
fields_symbol='TCLOSE,NEGOTIABLEMV,TOTMKTCAP,TURNRATE,PELFY,PETTM,PEMRQ,PELFYNPAAEI,PETTMNPAAEI'
start_date0='2016-01-01'
end_date0='2016-06-01'

hs300=get_constituents(index=index_symbol, fields='symbol, weight', df=True)
hs300

#注意最高40000条记录的限制
ex2=get_fundamentals(table=table_caiwu, symbols=hs300.symbol.tolist(), start_date=start_date0, end_date=end_date0,
                 fields=fields_symbol, df=True,limit=40000)
ex3=ex2.set_index(["end_date","symbol"])
history_data = history(symbol=hs300.symbol.tolist(), frequency='1d', start_time=start_date0,  end_time=end_date0, fields='close,eob,symbol', adjust=ADJUST_PREV, df= True)

history_data_u=history_data.set_index(["eob","symbol"]).unstack().close
history_data_p=history_data_u/history_data_u.shift(1)-1

h_m=history_data_p.shift(-1).stack()


for i in ex3.columns[1:]:
    z1=winsorize(ex3[i], limits=0.03)
    ex3[str(i)+"_s"]=standardize_series(z1)
    
f_y_1=pd.concat([h_m,ex3],axis=1).dropna()

for i in f_y_1.columns[11:]:
    print(i,spearmanr(f_y_1.iloc[:,0],f_y_1[i]))

TCLOSE_s SpearmanrResult(correlation=0.0024304070380963302, pvalue=0.7247354747216616)
NEGOTIABLEMV_s SpearmanrResult(correlation=-0.030925994390608693, pvalue=7.399044672333684e-06)
TOTMKTCAP_s SpearmanrResult(correlation=-0.03365402703222736, pvalue=1.0747022097519771e-06)
TURNRATE_s SpearmanrResult(correlation=-0.03952031836447587, pvalue=1.0147006651214881e-08)
PELFY_s SpearmanrResult(correlation=-0.002518884387190176, pvalue=0.715143939665623)
PETTM_s SpearmanrResult(correlation=-0.0035982048599351017, pvalue=0.6021313412906157)
PEMRQ_s SpearmanrResult(correlation=-0.00016186368529837246, pvalue=0.9812896565060665)
PELFYNPAAEI_s SpearmanrResult(correlation=0.0013786251791810065, pvalue=0.8416790911062658)
PETTMNPAAEI_s SpearmanrResult(correlation=-0.0008896404229984172, pvalue=0.8974382427465999)


## 单因子策略框架


### classwork 2
* 小市值策略“TOTMKTCAP”
* 策略运行时间2013-01-01到2014-12-30

https://www.bilibili.com/video/BV1uq4y177uX/

In [47]:
from datetime import timedelta

In [48]:
# 获取筛选时间：date1表示当前日期之前的100天，date2表示当前时间
date1 = (context.now - timedelta(days=100)).strftime("%Y-%m-%d %H:%M:%S")
date2 = context.now.strftime("%Y-%m-%d %H:%M:%S")

# 通过get_instruments获取所有的上市股票代码
all_stock = get_instruments(exchanges='SHSE, SZSE', sec_types=[1], fields='symbol, listed_date, delisted_date',
                            df=True)

# 剔除停牌和st股和上市不足100日的新股和退市股和B股
code = all_stock[(all_stock['listed_date'] < date1) & (all_stock['delisted_date'] > date2) &
                 (all_stock['symbol'].str[5] != '9') & (all_stock['symbol'].str[5] != '2')]

NameError: name 'context' is not defined

## 多因子策略框架

伟文同学的因子：表：'deriv_finance_indicator'   此三因子'ROEANNUAL,NPGRT,OPNCFPS' 最大的

### classwork 3

* 通过get_fundamentals_n获取2020-01-01最近一期的沪深300的'ROEANNUAL,NPGRT,OPNCFPS'，然后分别对此三个因子排序，以排序序号作为加权因子，选择三因子加权求和得到最大的前10只股票

* 在掘金中实现以上策略，时间选择2020-01-01到2021-12-30， 选股周期为1个月

In [18]:
ex=get_fundamentals_n(table='deriv_finance_indicator', symbols=hs300.symbol.tolist(), end_date='2020-01-01', count=1,
                   fields='ROEANNUAL,NPGRT,OPNCFPS',df=True)
ex

,symbol,pub_date,end_date,ROEANNUAL,NPGRT,OPNCFPS
0,SHSE.601966,2020-04-14 00:00:00+08:00,2019-12-31 00:00:00+08:00,16.2787,41.2040,2.3332
1,SZSE.002594,2020-04-22 00:00:00+08:00,2019-12-31 00:00:00+08:00,3.1146,-41.9303,5.4033
2,SHSE.688036,2020-04-28 00:00:00+08:00,2019-12-31 00:00:00+08:00,29.4702,172.7963,5.0506
3,SHSE.603019,2020-03-28 00:00:00+08:00,2019-12-31 00:00:00+08:00,15.1879,37.8603,3.5252
4,SHSE.601985,2020-04-24 00:00:00+08:00,2019-12-31 00:00:00+08:00,9.7984,0.4380,1.7131
...,...,...,...,...,...,...
295,SHSE.688009,2020-03-26 00:00:00+08:00,2019-12-31 00:00:00+08:00,11.8454,11.9502,0.3184
296,SZSE.002568,2020-03-31 00:00:00+08:00,2019-12-31 00:00:00+08:00,15.3137,142.6697,1.0206
297,SZSE.002624,2020-04-24 00:00:00+08:00,2019-12-31 00:00:00+08:00,16.7407,-11.9163,1.5647
298,SHSE.601898,2020-03-21 00:00:00+08:00,2019-12-31 00:00:00+08:00,5.9465,67.8984,1.6578


In [27]:
ex.set_index("symbol",inplace=True)

In [36]:
f1=ex.sort_values("ROEANNUAL",ascending=False)
f1["s1"]=list(range(1,len(f1)+1))

In [29]:
f1

,pub_date,end_date,ROEANNUAL,NPGRT,OPNCFPS,s1
symbol,,,,,,
SZSE.002466,2020-04-29 00:00:00+08:00,2019-12-31 00:00:00+08:00,-69.9844,-371.9572,1.5943,1
SZSE.002044,2020-04-30 00:00:00+08:00,2019-12-31 00:00:00+08:00,-12.4836,-205.5841,0.3213,2
SHSE.688561,2020-05-11 00:00:00+08:00,2019-12-31 00:00:00+08:00,-10.6215,43.2246,-1.9283,3
SZSE.000625,2020-04-30 00:00:00+08:00,2019-12-31 00:00:00+08:00,-5.8638,-488.8069,0.8083,4
SHSE.688126,2020-03-31 00:00:00+08:00,2019-12-31 00:00:00+08:00,-2.1263,-902.3971,0.4767,5
...,...,...,...,...,...,...
SHSE.603195,2020-04-27 00:00:00+08:00,2019-12-31 00:00:00+08:00,52.3640,37.3829,4.2543,296
SZSE.002607,2020-03-10 00:00:00+08:00,2019-12-31 00:00:00+08:00,56.5202,56.5243,0.4011,297
SZSE.300896,2020-06-30 00:00:00+08:00,2019-12-31 00:00:00+08:00,57.7899,148.6836,3.4413,298


In [37]:
f2=ex.sort_values("NPGRT",ascending=False)
f2["s2"]=list(range(1,len(f2)+1))
f3=ex.sort_values("OPNCFPS",ascending=False)
f3["s3"]=list(range(1,len(f3)+1))

In [38]:
f5=f1.s1+f2.s2+f3.s3

In [44]:
f5.sort_values()[0:10].index

Index(['SHSE.688169', 'SHSE.603160', 'SZSE.002714', 'SHSE.600132',
       'SHSE.688036', 'SZSE.300896', 'SZSE.300498', 'SHSE.605499',
       'SZSE.000661', 'SHSE.600346'],
      dtype='object', name='symbol')

In [46]:
#f5.sort_values()[0:10]

In [ ]:
    # ex=get_fundamentals_n(table='deriv_finance_indicator', symbols=code.symbol.tolist(), end_date=context.now, count=1,
    #                fields='ROEANNUAL,NPGRT,OPNCFPS',df=True)

    # ex.set_index("symbol",inplace=True)
    # f1=ex.sort_values("ROEANNUAL",ascending=False)
    # f1["s1"]=list(range(1,len(f1)+1))
    # f2=ex.sort_values("NPGRT",ascending=False)
    # f2["s2"]=list(range(1,len(f2)+1))
    # f3=ex.sort_values("OPNCFPS",ascending=False)
    # f3["s3"]=list(range(1,len(f3)+1))
    # f5=f1.s1+f2.s2+f3.s3
    # to_buy=f5.sort_values()[0:10].index.tolist()